In [ ]:
import functions_framework
from datetime import datetime, timedelta
from pytz import timezone
import pandas as pd
import sqlalchemy
import requests


@functions_framework.http
def insert(request):
    list_of_icao= ['EDDB', 'EDDH', 'EDDM']
    city_id_aerport= {
    'EDDB': 1,
    'EDDH': 2,
    'EDDM': 3
    }
    connection_string = connection()
    flights_df = get_flight_func(list_of_icao)
    insert_into_flights_arrivals(connection_string, flights_df)
    return 'Data successfully added'


def get_flight_func(list_of_icao):
    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))

    flight_items = []

    for icao in list_of_icao:
       
        times = [["00:00", "11:59"], ["12:00", "23:59"]]

        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            querystring = {
                "withLeg": "true",
                "direction": "Arrival",
                "withCancelled": "false",
                "withCodeshared": "true",
                "withCargo": "false",
                "withPrivate": "false"
            }

            headers = {
                "X-RapidAPI-Key": 'Introduce your APi Key here',  # Replace with your actual API key
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }

            response = requests.request("GET", url, headers=headers, params=querystring)
            flights_json = response.json()
            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

            for item in flights_json["arrivals"]:
                flight_item = {
                    "arrival_airport_icao": icao,
                    "departure_airport_icao": item["departure"]["airport"].get("icao", None),
                    "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
                    "flight_number": item.get("number", None),
                    "date_retrieved_at": retrieval_time
                }

                flight_items.append(flight_item)

    flights_df = pd.DataFrame(flight_items)
    flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6]
    flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"])
    flights_df["date_retrieved_at"] = pd.to_datetime(flights_df["date_retrieved_at"])

    return flights_df


def connection():
  connection_name = "burnished-flare-410609:europe-west1:wbs-mysql-db"
  db_user = "root"
  db_password = "Introduce your key here"
  schema_name = "gans"

  driver_name = 'mysql+pymysql'
  query_string = dict({"unix_socket": f"/cloudsql/{connection_name}"})

  db = sqlalchemy.create_engine(
      sqlalchemy.engine.url.URL(
          drivername = driver_name,
          username = db_user,
          password = db_password,
          database = schema_name,
          query = query_string,
      )
  )
  return db

def insert_into_flights_arrivals(connection_string, flights_df):
    return flights_df.to_sql(name="flights_arrivals", con=connection_string, if_exists='append', index=False)




In [ ]:
functions-framework==3.*
SQLAlchemy==1.4.37
PyMySQL==1.0.2
pandas==1.5.2
requests==2.31.0
pytz

In [ ]:
import functions_framework
import pandas as pd
import sqlalchemy
import requests


@functions_framework.http
def insert(request):
    list_of_cities = ['Berlin', 'Hamburg', 'Munich']
    city_id_map = {
    'Berlin': 1,
    'Hamburg': 2,
    'Munich': 3
    }
    connection_string = connection()
    weather_df = weather_func(list_of_cities, city_id_map)
    insert_into_weather(connection_string, weather_df)
    return 'Data successfully added'


def weather_func(cities, city_id_map):
    API_key = 'Introduce your API Key here'
    all_rows = []

    for city in cities:
        city_id = city_id_map.get(city)
        if city_id is None:
            continue  # Skip if city_id is not found

        weather = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
        weather_json = weather.json()

        for item in weather_json['list']:
            row = {
                'city_id': city_id,
                'datetime': item['dt_txt'],
                'temp': item.get('main', {}).get('temp'),
                'weather_main': item['weather'][0].get('main') if 'weather' in item and item['weather'] else None,
                'weather_description': item['weather'][0].get('description') if 'weather' in item and item['weather'] else None,
                'rain_in_last_3h': item.get('rain', {}).get('3h', 0),
                'clouds_all': item.get('clouds', {}).get('all'),
                'wind_speed': item.get('wind', {}).get('speed'),
                'wind_deg': item.get('wind', {}).get('deg')
            }
            all_rows.append(row)

    weather_df = pd.DataFrame(all_rows)
    return weather_df

def connection():
  connection_name = "burnished-flare-410609:europe-west1:wbs-mysql-db"
  db_user = "root"
  db_password = "Introduce your password here"
  schema_name = "gans"

  driver_name = 'mysql+pymysql'
  query_string = dict({"unix_socket": f"/cloudsql/{connection_name}"})

  db = sqlalchemy.create_engine(
      sqlalchemy.engine.url.URL(
          drivername = driver_name,
          username = db_user,
          password = db_password,
          database = schema_name,
          query = query_string,
      )
  )
  return db

def insert_into_weather(connection_string, weather_df):
    return weather_df.to_sql(name="weather", con=connection_string, if_exists='append', index=False)


In [ ]:
functions-framework==3.*
SQLAlchemy==1.4.37
PyMySQL==1.0.2
pandas==1.5.2
requests==2.31.0